# Automatic Differentiation (AD)

We all know how to take derivatives.
```julia
f(x) = 5*x^2 + 3

df(x) = 10*x

ddf(x) = 10
```

The promise of AD is

```julia
df(x) = derivative(f, x)

ddf(x) = derivative(df, x)
```

### What AD is not

**Symbolic differentiation:**
$$ \frac{d}{dx}x^n = n x^{n-1}. $$

**Numerical differentiation:**
$$ \frac{df}{dx} \approx \frac{f(x+h) - f(x)}{\Delta h} $$

## Forward mode AD

Key to AD is the application of the chain rule
$$\dfrac{d}{dx} f(g(x)) = \dfrac{df}{dg} \dfrac{dg}{dx}$$

Consider the function $f(a,b) = \ln(ab + \sin(a))$.

In [1]:
f(a,b) = log(a*b + sin(a))

f (generic function with 1 method)

In [2]:
f_derivative(a,b) = 1/(a*b + sin(a)) * (b + cos(a))

f_derivative (generic function with 1 method)

In [3]:
a = 3.1
b = 2.4
f_derivative(a,b)

0.18724182935843758

Dividing the function into the elementary steps, it corresponds to the following "*computational graph*":

<img src="comp_graph.svg" width=300px>

In [4]:
function f_graph(a,b)
    c1 = a*b
    c2 = sin(a)
    c3 = c1 + c2
    c4 = log(c3)
end

f_graph (generic function with 1 method)

In [5]:
f(a,b) == f_graph(a,b)

true

To calculate $\frac{\partial f}{\partial a}$ we have to apply the chain rule multiple times.

$\dfrac{\partial f}{\partial a} = \dfrac{\partial f}{\partial c_4} \dfrac{\partial c_4}{\partial a} = \dfrac{\partial f}{\partial c_4} \left( \dfrac{\partial c_4}{\partial c_3} \dfrac{\partial c_3}{\partial a}  \right) = \dfrac{\partial f}{\partial c_4} \left( \dfrac{\partial c_4}{\partial c_3} \left( \dfrac{\partial c_3}{\partial c_2} \dfrac{\partial c_2}{\partial a} + \dfrac{\partial c_3}{\partial c_1} \dfrac{\partial c_1}{\partial a}\right)  \right)$

In [8]:
function f_graph_derivative(a,b)
    c1 = a*b
    c1_ϵ = b
    
    c2 = sin(a)
    c2_ϵ = cos(a)
    
    c3 = c1 + c2
    c3_ϵ = c1_ϵ + c2_ϵ
    
    c4 = log(c3)
    c4_ϵ = 1/c3 * c3_ϵ
    
    c4, c4_ϵ
end

f_graph_derivative (generic function with 1 method)

In [9]:
f_graph_derivative(a,b)[2] == f_derivative(a,b)

true

**How can we automate this?**

In [6]:
# D for "dual number", invented by Clifford in 1873.
struct D <: Number
    x::Float64 # value
    ϵ::Float64 # derivative
end

In [7]:
import Base: +, *, /, -, sin, log, convert, promote_rule

a::D + b::D = D(a.x + b.x, a.ϵ + b.ϵ) # sum rule
a::D - b::D = D(a.x - b.x, a.ϵ - b.ϵ)
a::D * b::D = D(a.x * b.x, a.x * b.ϵ + a.ϵ * b.x) # product rule
a::D / b::D = D(a.x / b.x, (b.x * a.ϵ - a.x * b.ϵ)/b.x^2) # quotient rule
sin(a::D) = D(sin(a.x), cos(a.x) * a.ϵ)
log(a::D) = D(log(a.x), 1/a.x * a.ϵ)

Base.convert(::Type{D}, x::Real) = D(x, zero(x))
Base.promote_rule(::Type{D}, ::Type{<:Number}) = D

In [12]:
f(D(a,1), b)

D(2.0124440881688996, 0.18724182935843758)

Boom! That was easy!

In [13]:
f(D(a,1), b).ϵ ≈ f_derivative(a,b)

true

**How does this work?!**

The trick of forward mode AD is to make the computer do the rewrite `f -> f_graph_derivative` for you (and then optimize the resulting code structure).

In [16]:
@code_typed f(D(a,1), b)

CodeInfo(
1 ─ %1  = Base.getfield(a, :x)::Float64
│   %2  = Base.mul_float(%1, b)::Float64
│   %3  = Base.getfield(a, :x)::Float64
│   %4  = Base.mul_float(%3, 0.0)::Float64
│   %5  = Base.getfield(a, :ϵ)::Float64
│   %6  = Base.mul_float(%5, b)::Float64
│   %7  = Base.add_float(%4, %6)::Float64
│   %8  = Base.getfield(a, :x)::Float64
│   %9  = invoke Main.sin(%8::Float64)::Float64
│   %10 = Base.getfield(a, :x)::Float64
│   %11 = invoke Main.cos(%10::Float64)::Float64
│   %12 = Base.getfield(a, :ϵ)::Float64
│   %13 = Base.mul_float(%11, %12)::Float64
│   %14 = Base.add_float(%2, %9)::Float64
│   %15 = Base.add_float(%7, %13)::Float64
│   %16 = invoke Main.log(%14::Float64)::Float64
│   %17 = Base.div_float(1.0, %14)::Float64
│   %18 = Base.mul_float(%17, %15)::Float64
│   %19 = %new(Main.D, %16, %18)::D
└──       return %19
) => D

While this is somewhat hard to parse, plugging these operations manually into each other we find that this code equals

```julia
D.x = log(a.x*b + sin(a.x))
D.ϵ = 1/(a.x*b + sin(a.x)) * (a.x*0 + (a.ϵ*b) + cos(a.x)*a.ϵ)
```

which, if we drop `a.x*0`, set `a.ϵ = 1`, and rename `a.x` $\rightarrow$ `a`, reads

```julia
D.x = log(a*b + sin(a))
D.ϵ = 1/(a*b + sin(a)) * (b + cos(a)
```

This precisely matches our definitions from above:

```julia
f(a,b) = log(a*b + sin(a))

f_derivative(a,b) = 1/(a*b + sin(a)) * (b + cos(a))
```

Importantly, the compiler sees the entire "rewritten" code and can therefore apply optimizations. In this simple example, we find that the code produced by our simple Forward mode AD is essentially identical to the explicit implementation.

In [24]:
@code_llvm debuginfo=:none f_graph_derivative(a,b)


define void @julia_f_graph_derivative_17587([2 x double]* noalias nocapture sret, double, double) {
top:
  %3 = fmul double %1, %2
  %4 = call double @julia_sin_17494(double %1)
  %5 = call double @julia_cos_17497(double %1)
  %6 = fadd double %3, %4
  %7 = fadd double %5, %2
  %8 = call double @julia_log_11868(double %6)
  %9 = fdiv double 1.000000e+00, %6
  %10 = fmul double %9, %7
  %.sroa.0.0..sroa_idx = getelementptr inbounds [2 x double], [2 x double]* %0, i64 0, i64 0
  store double %8, double* %.sroa.0.0..sroa_idx, align 8
  %.sroa.2.0..sroa_idx1 = getelementptr inbounds [2 x double], [2 x double]* %0, i64 0, i64 1
  store double %10, double* %.sroa.2.0..sroa_idx1, align 8
  ret void
}


In [25]:
@code_llvm debuginfo=:none f(D(a,1), b)


define void @julia_f_17598({ double, double }* noalias nocapture sret, { double, double } addrspace(11)* nocapture nonnull readonly dereferenceable(16), double) {
top:
  %3 = getelementptr inbounds { double, double }, { double, double } addrspace(11)* %1, i64 0, i32 0
  %4 = load double, double addrspace(11)* %3, align 8
  %5 = fmul double %4, %2
  %6 = fmul double %4, 0.000000e+00
  %7 = getelementptr inbounds { double, double }, { double, double } addrspace(11)* %1, i64 0, i32 1
  %8 = load double, double addrspace(11)* %7, align 8
  %9 = fmul double %8, %2
  %10 = fadd double %6, %9
  %11 = call double @julia_sin_17494(double %4)
  %12 = call double @julia_cos_17497(double %4)
  %13 = fmul double %12, %8
  %14 = fadd double %5, %11
  %15 = fadd double %10, %13
  %16 = call double @julia_log_11868(double %14)
  %17 = fdiv double 1.000000e+00, %14
  %18 = fmul double %17, %15
  %.sroa.0.0..sroa_idx = getelementptr inbounds { double, double }, { double, double }* %0, i64 0, i32 0
  st

It's general:

In [ ]:
# utility function for our small forward AD
derivative(f::Function, x::Number) = f(D(x, one(x))).ϵ

In [ ]:
derivative(x->f(x,b), a)

In [ ]:
derivative(x->3*x^2+4x+5, 2)

In [ ]:
derivative(x->sin(x)*log(x), 3)

Or as a function:

In [ ]:
df(x) = derivative(a->f(a,b),x) # partial derivative wrt a

In [ ]:
df(1.23)

## Taking derivatives of code: Babylonian sqrt

> Repeat $t \leftarrow (t + x/2)/2$ until $t$ converges to $\sqrt{x}$.

In [3]:
@inline function Babylonian(x; N = 10)
    t = (1+x)/2
    for i = 2:N
        t = (t + x/t)/2
    end
    t
end

Babylonian (generic function with 1 method)

In [4]:
Babylonian(2), √2

(1.414213562373095, 1.4142135623730951)

In [ ]:
using Plots

xs = 0:0.01:49

p = plot(title = "Those Babylonians really knew what they were doing")
for i in 1:5
    plot!(p, xs, [Babylonian(x; N=i) for x in xs], label="Iteration $i")
end

plot!(p, xs, sqrt.(xs), label="sqrt", color=:black)

## ... and now the derivative, automagically

The same babylonian algorithm with no rewrite at all computes properly the derivative as the check shows.

In [5]:
Babylonian(D(5, 1))

LoadError: UndefVarError: D not defined

In [ ]:
√5, 0.5 / √5

It just works and is efficient.

In [ ]:
@code_native debuginfo=:none Babylonian(D(5, 1))

Recursion? Works as well...

In [ ]:
pow(x, n) = n <= 0 ? 1 : x*pow(x, n-1)

In [ ]:
derivative(x -> pow(x,3), 2)

Deriving our Vandermonde matrix from yesterday? Sure:

In [38]:
function vander_generic(x::AbstractVector{T}) where T
    m = length(x)
    V = Matrix{T}(undef, m, m)
    for j = 1:m
        V[j,1] = one(x[j])
    end
    for i= 2:m
        for j = 1:m
            V[j,i] = x[j] * V[j,i-1]
            end
        end
    return V
end

vander_generic (generic function with 1 method)

In [44]:
a, b, c, d = 2, 3, 4, 5
V = vander_generic([D(a,1), D(b,1), D(c,1), D(d,1)])

4×4 Array{D,2}:
 D(1.0, 0.0)  D(2.0, 1.0)    D(4.0, 4.0)    D(8.0, 12.0)
 D(1.0, 0.0)  D(3.0, 1.0)    D(9.0, 6.0)   D(27.0, 27.0)
 D(1.0, 0.0)  D(4.0, 1.0)   D(16.0, 8.0)   D(64.0, 48.0)
 D(1.0, 0.0)  D(5.0, 1.0)  D(25.0, 10.0)  D(125.0, 75.0)

In [47]:
(x->getfield(x, :ϵ)).(V)

4×4 Array{Float64,2}:
 0.0  1.0   4.0  12.0
 0.0  1.0   6.0  27.0
 0.0  1.0   8.0  48.0
 0.0  1.0  10.0  75.0

## Symbolically (because we can)

The below is mathematically equivalent, **though not what the computation is doing**.

In [2]:
using SymPy

In [8]:
x = symbols("x")

display("Iterations as a function of x")
for k = 1:5
    display(simplify(Babylonian(x; N=k)))
end

display("Derivatives as a function of x")
for k = 1:5
    display(simplify(diff(simplify(Babylonian(x; N=k)), x)))
end

"Iterations as a function of x"

x   1
─ + ─
2   2

           2
    (x + 1) 
x + ────────
       4    
────────────
   x + 1    

 4       3       2           
x  + 28⋅x  + 70⋅x  + 28⋅x + 1
─────────────────────────────
     ⎛ 3      2          ⎞   
   8⋅⎝x  + 7⋅x  + 7⋅x + 1⎠   

 8        7         6         5          4         3         2            
x  + 120⋅x  + 1820⋅x  + 8008⋅x  + 12870⋅x  + 8008⋅x  + 1820⋅x  + 120⋅x + 1
──────────────────────────────────────────────────────────────────────────
         ⎛ 7       6        5        4        3        2           ⎞      
      16⋅⎝x  + 35⋅x  + 273⋅x  + 715⋅x  + 715⋅x  + 273⋅x  + 35⋅x + 1⎠      

...

1/2

   2            
  x  + 2⋅x + 5  
────────────────
  ⎛ 2          ⎞
4⋅⎝x  + 2⋅x + 1⎠

 6       5        4        3        2             
x  + 14⋅x  + 147⋅x  + 340⋅x  + 375⋅x  + 126⋅x + 21
──────────────────────────────────────────────────
  ⎛ 6       5       4        3       2           ⎞
8⋅⎝x  + 14⋅x  + 63⋅x  + 100⋅x  + 63⋅x  + 14⋅x + 1⎠

 14       13         12          11           10            9            8    
x   + 70⋅x   + 3199⋅x   + 52364⋅x   + 438945⋅x   + 2014506⋅x  + 5430215⋅x  + 8
──────────────────────────────────────────────────────────────────────────────
      ⎛ 14       13         12          11           10           9           
   16⋅⎝x   + 70⋅x   + 1771⋅x   + 20540⋅x   + 126009⋅x   + 440986⋅x  + 920795⋅x

        7            6            5            4           3          2       
836200⋅x  + 8842635⋅x  + 5425210⋅x  + 2017509⋅x  + 437580⋅x  + 52819⋅x  + 3094
──────────────────────────────────────────────────────────────────────────────
8            7           6           5           4          3         2       
  + 1173960⋅x  + 920795⋅x  + 440986⋅x  + 126009⋅x  + 20540⋅x  + 1771⋅x  + 70⋅x

       
⋅x + 85
───────
    ⎞  
 + 1⎠  

 30        29          28            27              26               25      
x   + 310⋅x   + 59799⋅x   + 4851004⋅x   + 215176549⋅x   + 5809257090⋅x   + 102
──────────────────────────────────────────────────────────────────────────────
                     ⎛ 30        29          28            27             26  
                  32⋅⎝x   + 310⋅x   + 36611⋅x   + 2161196⋅x   + 73961629⋅x   +

           24                  23                   22                   21   
632077611⋅x   + 1246240871640⋅x   + 10776333438765⋅x   + 68124037776390⋅x   + 
──────────────────────────────────────────────────────────────────────────────
             25                24                 23                  22      
 1603620018⋅x   + 23367042639⋅x   + 238538538360⋅x   + 1758637118685⋅x   + 957

                 20                     19                     18             
321156247784955⋅x   + 1146261110726340⋅x   + 3133113888931089⋅x   + 6614351291
──────────────────────────────────────────────────

In [25]:
@code_native debuginfo=:none Babylonian(D(5, 1); N=5)

	.section	__TEXT,__text,regular,pure_instructions
	vmovupd	(%rdx), %xmm1
	movabsq	$5205905600, %rax       ## imm = 0x1364BD0C0
	vaddsd	(%rax), %xmm1, %xmm0
	vpermilpd	$1, %xmm1, %xmm2 ## xmm2 = xmm1[1,0]
	vxorpd	%xmm3, %xmm3, %xmm3
	vaddsd	%xmm3, %xmm2, %xmm2
	vaddsd	%xmm2, %xmm2, %xmm2
	vmulsd	%xmm3, %xmm0, %xmm3
	vsubsd	%xmm3, %xmm2, %xmm2
	vunpcklpd	%xmm0, %xmm2, %xmm0 ## xmm0 = xmm2[0],xmm0[0]
	movabsq	$5205905616, %rdx       ## imm = 0x1364BD0D0
	vmulpd	(%rdx), %xmm0, %xmm0
	movq	%rdi, %rax
	movq	(%rsi), %rsi
	testq	%rsi, %rsi
	movl	$1, %ecx
	cmovgq	%rsi, %rcx
	cmpq	$2, %rsi
	jl	L191
	decq	%rcx
	movabsq	$5205905608, %rsi       ## imm = 0x1364BD0C8
	vmovsd	(%rsi), %xmm2           ## xmm2 = mem[0],zero
	vmovapd	(%rdx), %xmm3
	nopl	(%rax,%rax)
L112:
	vmulpd	%xmm1, %xmm0, %xmm4
	vpermilpd	$1, %xmm4, %xmm5 ## xmm5 = xmm4[1,0]
	vsubsd	%xmm4, %xmm5, %xmm4
	vpermilpd	$1, %xmm0, %xmm5 ## xmm5 = xmm0[1,0]
	vmulsd	%xmm5, %xmm5, %xmm5
	vunpcklpd	%xmm1, %xmm4, %xmm4 ## xmm4 = xmm4[0],xmm1[0]
	

## ForwardDiff.jl

Now that we have understood how forward AD works, we can use the more feature complete package [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jl).

In [ ]:
using ForwardDiff

In [ ]:
ForwardDiff.derivative(Babylonian, 2)

In [ ]:
@edit ForwardDiff.derivative(Babylonian, 2)

(Note: [DiffRules.jl](https://github.com/JuliaDiff/DiffRules.jl))

## Reverse mode AD

Forward mode:
$\dfrac{\partial f}{\partial x} = \dfrac{\partial f}{\partial c_4} \dfrac{\partial c_4}{\partial x} = \dfrac{\partial f}{\partial c_4} \left( \dfrac{\partial c_4}{\partial c_3} \dfrac{\partial c_3}{\partial x}  \right) = \dfrac{\partial f}{\partial c_4} \left( \dfrac{\partial c_4}{\partial c_3} \left( \dfrac{\partial c_3}{\partial c_2} \dfrac{\partial c_2}{\partial x} + \dfrac{\partial c_3}{\partial c_1} \dfrac{\partial c_1}{\partial x}\right)  \right)$

Reverse mode:
$\dfrac{\partial f}{\partial x} = \dfrac{\partial f}{\partial c_4} \dfrac{\partial c_4}{\partial x} = \left( \dfrac{\partial f}{\partial c_3}\dfrac{\partial c_3}{\partial c_4}   \right) \dfrac{\partial c_4}{\partial x} = \left( \left( \dfrac{\partial f}{\partial c_2} \dfrac{\partial c_2}{\partial c_3} + \dfrac{\partial f}{\partial c_1} \dfrac{\partial c_1}{\partial c_3} \right) \dfrac{\partial c_3}{\partial c_4} \right) \dfrac{\partial c_4}{\partial x}$

Forward mode AD requires $n$ passes in order to compute an $n$-dimensional
gradient.

Reverse mode AD requires only a single run in order to compute a complete gradient but requires two passes through the graph: a forward pass during which necessary intermediate values are computed and a backward pass which computes the gradient.

*Rule of thumb:*

Forward mode is good for $\mathbb{R} \rightarrow \mathbb{R}^n$ while reverse mode is good for $\mathbb{R}^n \rightarrow \mathbb{R}$.

An efficient source-to-source reverse mode AD is implemented in [Zygote.jl](https://github.com/FluxML/Zygote.jl), the AD underlying [Flux.jl](https://fluxml.ai/) (since version 0.10).

In [ ]:
using Zygote

In [ ]:
f(x) = 5*x + 3

In [ ]:
gradient(f, 5)

In [ ]:
@code_llvm debuginfo=:none gradient(f,5)

## Some nice reads

Blog posts:

* ML in Julia: https://julialang.org/blog/2018/12/ml-language-compiler

* Nice example: https://fluxml.ai/2019/03/05/dp-vs-rl.html

* Nice interactive examples: https://fluxml.ai/experiments/

* Why Julia for ML? https://julialang.org/blog/2017/12/ml&pl

* Neural networks with differential equation layers: https://julialang.org/blog/2019/01/fluxdiffeq

* Implement Your Own Automatic Differentiation with Julia in ONE day : http://blog.rogerluo.me/2018/10/23/write-an-ad-in-one-day/

* Implement Your Own Source To Source AD in ONE day!: http://blog.rogerluo.me/2019/07/27/yassad/

Repositories:

* AD flavors, like forward and reverse mode AD: https://github.com/MikeInnes/diff-zoo (Mike is one of the smartest Julia ML heads)

Talks:

* AD is a compiler problem: https://juliacomputing.com/assets/pdf/CGO_C4ML_talk.pdf